<a href="https://colab.research.google.com/github/matthewlai12/ECE-Lab/blob/main/ECE449Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

model = models.Sequential()

#define filters and convolutional layers here

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

#Add a maxpooling layer
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#Flatten the output and give it to a fully conencted layer
model.add(layers.Flatten())

#One hidden layer maps the flattened neurons to output
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Load and preprocess the data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize the data
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255

# One-hot encoding of labels
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up stratified 5-fold cross-validation
skf = StratifiedKFold(n_splits=5)
accuracies = []

for train_index, val_index in skf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train_encoded[train_index], y_train_encoded[val_index]

    # Train the model
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=128, verbose=1)

    # Evaluate on validation fold
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracies.append(val_accuracy)

# Output results
print(f'Cross-validation accuracies: {accuracies}')
print(f'Mean accuracy: {np.mean(accuracies)}')

# Final training on the full training set and evaluation on the test set
model.fit(X_train, y_train_encoded, epochs=10, batch_size=128, verbose=1)
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {test_accuracy}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.7734 - loss: 0.8641
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9484 - loss: 0.1835
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9655 - loss: 0.1263
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 37ms/step - accuracy: 0.9750 - loss: 0.0942
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9789 - loss: 0.0777
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.9813 - loss: 0.0671
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 19s 35ms/step - accuracy: 0.9828 - loss: 0.0621
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9846 - loss: 0.0556
Epoch 9/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - accuracy: 0.9854 - loss: 0.0503
Epoch 10/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.9872 - loss: 0.0458
Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 15s 41ms/step - accuracy: 0.9844 - loss: 0.0544
Epoch 2/10
375/375 ━━━━━━━━━━━━

#**ECE 449 Lab 3: Matthew Lai**
In this lab, our goal is to use Convolutional Neural Networks (CNNs) on a well known dataset. The dataset is the MNIST digits database that is distibuted through with Keras. We are going to design and parameterize a basic CNN using a stratified cross-validation experimental approach. We will then evaluate the performance of this CNN on a predetermined out-of-sample test set.

##Data Collection